https://www.youtube.com/watch?v=tcqEUSNCn8I
https://github.com/pixegami/langchain-rag-tutorial/blob/main/query_data.py

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import os
import shutil
from langchain_community.llms import Ollama
import json

In [3]:
paper_id = "2402.01383v1"
#CHROMA_PATH = paper_id+"/chroma"
CHROMA_PATH = paper_id+"/chroma300"
DATA_PATH = paper_id

In [4]:

def store_summary(survey1,filename):
    with open(f'summaries/'+filename+'.txt', 'w') as file:
        file.write(survey1)
def write_json(documents_data,file_name):
    converted_data = [dict(item) for item in documents_data['source_documents']]
    dict_data = {'answer':documents_data['result'],'source':converted_data}
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


### Create Chroma database

In [5]:

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

In [7]:
documents = load_documents()

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
documents[4].metadata

{'source': '2402.01383v1/2307.07889v3.LLM_Comparative_Assessment__Zero_shot_NLG_Evaluation_through_Pairwise_Comparisons_using_Large_Language_Models.pdf'}

In [9]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [10]:
chunks = split_text(documents)

Split 45 documents into 20318 chunks.
language processing benchmarks. In response to this challenge, recent studies commonly introduce an evaluation approach, namely the Elo rating sys- tem (Elo, 1967), wherein either human or LLM judges are enlisted to adjudicate between two LLM- generated outputs (Askell et al., 2021; Bai et al.,
{'source': '2402.01383v1/2307.03025v3.Style_Over_Substance__Evaluation_Biases_for_Large_Language_Models.pdf', 'start_index': 2067}


In [11]:
#chunks[11].page_content
some = chunks[0:10]

In [7]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [14]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
     #   shutil.rmtree(CHROMA_PATH)


    # Ensure the directory exists and has the correct permissions
    os.makedirs(CHROMA_PATH, exist_ok=True)
   # os.chmod(CHROMA_PATH, 0o755)
        
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [16]:
save_to_chroma(chunks)

Saved 20318 chunks to 2402.01383v1/chroma300.


/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Use database

In [8]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [18]:
from langchain.chains import RetrievalQA

In [19]:
llm = Ollama(model = "mistral",temperature=0)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(),
            return_source_documents = True,
            
            #memory=ConversationSummaryMemory(llm = Ollama(model="mistral")),
            #chain_type_kwargs={"prompt": pt, "verbose": True},
        )

answer = rag.invoke("What is the recent development of LLM generated text evaluation?")
rag_overview = answer['result']

In [20]:
print(rag_overview)
store_summary(rag_overview,'rag_overview300')

 Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, and there are limitations that provide opportunities for future work, such as the performan

In [21]:
a = answer['source_documents'][0]
converted_data = [dict(item) for item in answer['source_documents']]
#dict(answer['source_documents'])
converted_data
answer

{'query': 'What is the recent development of LLM generated text evaluation?',
 'result': " Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, a

In [45]:
write_json(answer,'rag_overview300')

In [48]:
type(answer)
sources = [doc.metadata.get("source", None) for doc, _score in answer]
sources

ValueError: too many values to unpack (expected 2)

### Change more parameters example

In [22]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


retriever=db.as_retriever(20)
llm = Ollama(model = "mistral",temperature=0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)


TypeError: VectorStore.as_retriever() takes 1 positional argument but 2 were given

In [26]:
query = 'What is the recent development of LLM generated text evaluation?'
chain.invoke({"input": query})

TypeError: 'VectorStoreRetriever' object is not callable

### Other parameters
https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339

In [13]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
Write your answer in about 2000 words.
"""

In [28]:
def query_rag(query_text,k=3):
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=k)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  llm = Ollama(model = "mistral",temperature=0)
  # Generate response text based on the prompt
  response_text = llm.invoke(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources} \nSourceText:{context_text}"
  return formatted_response, response_text



In [29]:
topic = "LLM-based NLG Evaluation: Current Status and Challenges"
db.similarity_search_with_relevance_scores(f'what are recent developments in {topic}?', k=10)

[(Document(metadata={'source': '2402.01383v1/2312.10355v1.CoAScore__Chain_of_Aspects_Prompting_for_NLG_Evaluation.pdf', 'start_index': 16491}, page_content='aspect. This approach showcases the versatility and potential of LLMs in improving NLG evaluation methodologies.'),
  0.8336756809872163),
 (Document(metadata={'source': '2402.01383v1/2312.10355v1.CoAScore__Chain_of_Aspects_Prompting_for_NLG_Evaluation.pdf', 'start_index': 13014}, page_content='prompts often lead to relatively signifi- cant differences in the evaluation results [4, 12, 15, 36]. Additionally, there is ongoing research into leveraging LLMs for the generation of NLG evaluation datasets, aiming to reduce the need for manual evaluations to some extent [5, 25]. In our work, we'),
  0.81348304157195),
 (Document(metadata={'source': '2402.01383v1/2312.10355v1.CoAScore__Chain_of_Aspects_Prompting_for_NLG_Evaluation.pdf', 'start_index': 493}, page_content='superior performance on various NLG evaluation tasks. However, curren

In [30]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=10)
# and finally, inspect our final response!
print(response_text)

 Recent developments in Large Language Models (LLMs) have significantly impacted the field of Natural Language Generation (NLG), leading to new research directions and challenges in NLG evaluation. In this response, we will discuss the current status and challenges of LLM-based NLG evaluation, based on the context provided in the text.

Firstly, it is essential to understand that NLG evaluation is a long-standing task in Natural Language Processing (NLP), which has become more challenging with the rapid development of LLMs (Celikyilmaz et al., 2020; Chang et al., 2023). Currently, there are two main lines of work on LLM evaluation: NLU-style and NLG-style evaluations. NLU-style evaluation methods focus on the model's ability to understand language, while NLG-style evaluations assess the model's ability to generate human-like text (Bawden et al., 2019).

Recent research works have focused on LLM-based evaluators due to their promising instruction-following and generalization capability.

In [31]:
store_summary(formatted_response,'rag10')

In [32]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=5)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag5')

 Recent developments in Large Language Models (LLMs) have significantly impacted the field of Natural Language Generation (NLG), leading to new research directions and challenges in evaluating NLG systems using LLMs. In this response, we will discuss the current status and challenges of LLM-based NLG evaluation.

Firstly, it is essential to understand that LLMs have shown remarkable performance on various NLG evaluation tasks (Liu et al., 2021; Raffel et al., 2019). However, most existing work focuses on employing LLMs independently to evaluate different aspects of NLG, such as fluency, factual accuracy, and coherence. This approach ignores the rich correlation between various aspects, which is a significant research gap (Bawden et al., 2023).

One line of recent work on LLM-based NLG evaluation focuses on preliminary explorations of LLM-based evaluators using prompting methods (Schick et al., 2021; Keskar et al., 2022). These studies aim to leverage the instruction-following and gener

In [33]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=8)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag8')

 Recent developments in Large Language Models (LLMs) have significantly impacted the field of Natural Language Generation (NLG), leading to new research directions and challenges in evaluating NLG systems using LLMs. In this response, we will discuss the current status and challenges of LLM-based NLG evaluation.

Firstly, it is essential to understand that NLG evaluation is a long-standing task in Natural Language Processing (NLP), which has become more challenging with the rapid development of LLMs (Celikyilmaz et al., 2020; Chang et al., 2023). Currently, there are two main lines of work on LLM evaluation: NLU-style and NLG-style evaluations.

NLU-style evaluation methods focus on measuring the ability of an LLM to understand and process natural language instructions accurately (Brown et al., 1995). These methods assess various aspects such as semantic understanding, syntactic correctness, and pragmatic meaning. However, NLU-style evaluations may not fully capture the nuances of open